In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Colon_and_Rectal_Cancer"
cohort = "GSE56699"

# Input paths
in_trait_dir = "../../input/GEO/Colon_and_Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Colon_and_Rectal_Cancer/GSE56699"

# Output paths
out_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/GSE56699.csv"
out_gene_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/gene_data/GSE56699.csv"
out_clinical_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/clinical_data/GSE56699.csv"
json_path = "../../output/preprocess/Colon_and_Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Stromal contribution to expression signatures of colorectal cancer (rectal samples)"
!Series_summary	"We noticed that a recently identified poor prognosis stem/serrated molecular subtype of colorectal cancer (CRC) is characterized by up-regulation of transcripts known to be also expressed by stromal cells. To better define the origin of such transcripts, we analyzed  RNAseq and microarray datasets from CRC mouse xenografts, where human cancer cells are supported by murine stroma. The analysis revealed that mRNA levels of stem/serrated subtype genes are mostly due to stromal expression, even when the stromal fraction is below 5%. Indeed, a classifier based on genes exclusively expressed by cancer-associated fibroblasts was significantly associated, in multiple datasets, to poor prognosis of CRC and to radioresistance of rectal cancer."
!Series_summary	""
!Series_summary	""
!Series_overall_design	"Molecular Characterization of 72 primary rectal canc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Analyze Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from rectal cancer specimens
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Analyzing the sample characteristics dictionary:

# For trait (Colon_and_Rectal_Cancer):
# Row 4 has "response 3 classes" which indicates cancer response to treatment
trait_row = 4  

# For age:
# No explicit age information in the sample characteristics
age_row = None

# For gender:
# No gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert cancer response value to binary format (0 for complete response, 1 for others)."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon and strip whitespace
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert values based on "response 3 classes"
    if value == "CR":  # Complete Response
        return 0
    elif value in ["RES", "PR"]:  # Resistant or Partial Response
        return 1
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous format."""
    # This function is defined but not used since age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary format (0 for female, 1 for male)."""
    # This function is defined but not used since gender data is not available
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # We need to create the clinical data DataFrame from the sample characteristics dictionary
    # The sample characteristics dictionary is provided in the previous output
    
    # Create a dictionary mapping sample IDs to their characteristics
    sample_ids = ['GSM1369681', 'GSM1369682', 'GSM1369683', 'GSM1369684', 'GSM1369685',
                  'GSM1369686', 'GSM1369687', 'GSM1369688', 'GSM1369689', 'GSM1369690',
                  'GSM1369691', 'GSM1369692', 'GSM1369693', 'GSM1369694', 'GSM1369695',
                  'GSM1369696', 'GSM1369697', 'GSM1369698', 'GSM1369699', 'GSM1369700',
                  'GSM1369701', 'GSM1369702', 'GSM1369703', 'GSM1369704', 'GSM1369705',
                  'GSM1369706', 'GSM1369707', 'GSM1369708', 'GSM1369709', 'GSM1369710']
    
    # Create the clinical data DataFrame with sample characteristics
    clinical_data = pd.DataFrame(index=sample_ids)
    
    # Fill in the trait values from row 4
    trait_values = [
        'response 3 classes: CR',
        'response 3 classes: RES',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR',
        'response 3 classes: PR'
    ]
    
    # Add traits to the DataFrame
    clinical_data[trait_row] = trait_values
    
    # Extract clinical features using the function from the library
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of selected clinical features:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the extracted clinical features to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical features:
{4: [1.0]}
Clinical data saved to: ../../output/preprocess/Colon_and_Rectal_Cancer/clinical_data/GSE56699.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 79
Header line: "ID_REF"	"GSM1366951"	"GSM1366952"	"GSM1366953"	"GSM1366954"	"GSM1366955"	"GSM1366956"	"GSM1366957"	"GSM1366958"	"GSM1366959"	"GSM1366960"	"GSM1366961"	"GSM1366962"	"GSM1366963"	"GSM1366964"	"GSM1366965"	"GSM1366966"	"GSM1366967"	"GSM1366968"	"GSM1366969"	"GSM1366970"	"GSM1366971"	"GSM1366972"	"GSM1366973"	"GSM1366974"	"GSM1366975"	"GSM1366976"	"GSM1366977"	"GSM1366978"	"GSM1366979"	"GSM1366980"	"GSM1366981"	"GSM1366982"	"GSM1366983"	"GSM1366984"	"GSM1366985"	"GSM1366986"	"GSM1366987"	"GSM1366988"	"GSM1366989"	"GSM1366990"	"GSM1366991"	"GSM1366992"	"GSM1366993"	"GSM1366994"	"GSM1366995"	"GSM1366996"	"GSM1366997"	"GSM1366998"	"GSM1366999"	"GSM1367000"	"GSM1367001"	"GSM1367002"	"GSM1367003"	"GSM1367004"	"GSM1367005"	"GSM1367006"	"GSM1367007"	"GSM1367008"	"GSM1367009"	"GSM1367010"	"GSM1367011"	"GSM1367012"	"GSM1367013"	"GSM1367014"	"GSM1367015"	"GSM1367016"	"GSM1367017"	"GSM1367018"	"GSM1367019"	"GSM1367020"	"GSM1367021"	"GSM1367022"
First data li

Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers, they start with "ILMN_" which indicates they are Illumina probe IDs
# These are not standard human gene symbols and will need to be mapped to gene symbols

# Illumina IDs (ILMN_) are microarray probe identifiers from Illumina BeadArray platforms
# They need to be converted to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Addres

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene annotation and expression data to determine the mapping columns
# Based on the output, 'ID' in gene_annotation corresponds to 'ILMN_xxxxx' identifiers in gene_data
# 'Symbol' is the column containing gene symbols in gene_annotation

# 2. Get gene mapping dataframe by extracting ID and Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="Symbol")

print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This will handle the many-to-many relations by dividing probe values equally among mapped genes
# and then summing up all probe values for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print(f"Dimensions of gene expression data after mapping: {gene_data.shape}")
print("Preview of gene expression data after mapping (first 5 genes):")
print(gene_data.head())

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping preview (first 5 rows):
             ID        Gene
0  ILMN_3166687  ERCC-00162
1  ILMN_3165566  ERCC-00071
2  ILMN_3164811  ERCC-00009
3  ILMN_3165363  ERCC-00053
4  ILMN_3166511  ERCC-00144


Dimensions of gene expression data after mapping: (20211, 72)
Preview of gene expression data after mapping (first 5 genes):
       GSM1366951   GSM1366952  GSM1366953  GSM1366954   GSM1366955  \
Gene                                                                  
A1BG   118.389670   144.600702  122.886608  133.022756   142.476847   
A1CF   759.060159  1817.612371  808.235136  977.351679  1207.345218   
A26C3  113.607415   106.275437  123.190609  119.261428   116.022301   
A2BP1  531.492788   700.987161  602.783304  657.927725   789.555525   
A2LD1  148.447316   153.643459  300.927913  420.407326   164.404106   

        GSM1366956   GSM1366957   GSM1366958   GSM1366959  GSM1366960  ...  \
Gene                                                                   ...   
A1BG    131.775939   128.508176   135.696979   119.674105  129.621366  ...   
A1CF   1821.464618  1854.497192  1744.429754   770.568048  488.976178  ...   
A26C3   105.967047   110.855337   112.252253   116.457441  112.15

Gene expression data saved to: ../../output/preprocess/Colon_and_Rectal_Cancer/gene_data/GSE56699.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the normalized gene data 
gene_data = pd.read_csv(out_gene_data_file, index_col=0)

# 2. Re-extract clinical features from the SOFT file to get proper clinical data
# Use the actual clinical data from the matrix file properly
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# 3. Create a correct clinical features dataframe
# First inspect what's in the clinical data
clinical_data_dict = get_unique_values_by_row(clinical_data)
print("Clinical data structure:")
print(clinical_data_dict)

# Based on the sample characteristics dictionary shown previously, 
# extract and process clinical features
selected_clinical_df = pd.DataFrame()

# Process disease state row manually to ensure correct mapping
disease_row = clinical_data.iloc[trait_row]
samples = [col for col in disease_row.index if col != "!Sample_geo_accession"]
trait_values = []

for sample in samples:
    value = disease_row[sample]
    if pd.isna(value):
        trait_values.append(None)
    else:
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "IBS" in value:
            trait_values.append(1)  # IBS is our target trait
        elif "IBD" in value:
            trait_values.append(0)  # IBD is the control
        else:
            trait_values.append(None)

# Create dataframe with processed values
selected_clinical_df[trait] = trait_values
selected_clinical_df.index = samples

# Save the corrected clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Corrected clinical data saved to {out_clinical_data_file}")

# 4. Link the clinical and genetic data
linked_data = pd.DataFrame()
# Transpose gene data to have samples as rows and genes as columns
gene_data_t = gene_data.T
# Verify alignment of sample IDs between clinical and gene data
common_samples = list(set(selected_clinical_df.index) & set(gene_data_t.index))
if common_samples:
    gene_data_filtered = gene_data_t.loc[common_samples]
    clinical_data_filtered = selected_clinical_df.loc[common_samples]
    # Join the data
    linked_data = pd.concat([clinical_data_filtered, gene_data_filtered], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Alternative linking approach if sample IDs don't directly match
    print("No common sample IDs found. Attempting alternative linking...")
    # The GSM ids in gene data columns may correspond to the sample IDs
    clinical_data_reset = selected_clinical_df.reset_index()
    clinical_data_reset.columns = ["Sample"] + list(clinical_data_reset.columns[1:])
    gene_data_cols = list(gene_data.columns)
    
    # Create merged dataframe
    data_dict = {trait: []}
    # Add trait values
    for col in gene_data_cols:
        sample_idx = clinical_data_reset.index[clinical_data_reset["Sample"] == col] if "Sample" in clinical_data_reset.columns else []
        if len(sample_idx) > 0:
            data_dict[trait].append(clinical_data_reset.loc[sample_idx[0], trait])
        else:
            data_dict[trait].append(None)
    
    # Add gene expression values
    for gene in gene_data.index:
        data_dict[gene] = list(gene_data.loc[gene])
    
    linked_data = pd.DataFrame(data_dict, index=gene_data_cols)
    print(f"Alternative linked data shape: {linked_data.shape}")

# 5. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 6. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 7. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from patients with IBS and IBD, examining effects of relaxation response mind-body intervention."
)

# 8. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Clinical data structure:
{0: ['patient code: RCa0001', 'patient code: RCa0002', 'patient code: RCa0003', 'patient code: RCa0004', 'patient code: RCa0005', 'patient code: RCa0006', 'patient code: RCa0007', 'patient code: RCa0008', 'patient code: RCa0009', 'patient code: RCa0010', 'patient code: RCa0011', 'patient code: RCa0012', 'patient code: RCa0013', 'patient code: RCa0014', 'patient code: RCa0015', 'patient code: RCa0016', 'patient code: RCa0024', 'patient code: RCa0026', 'patient code: RCa0027', 'patient code: RCa0028', 'patient code: RCa0029', 'patient code: RCa0030', 'patient code: RCa0031', 'patient code: RCa0032', 'patient code: RCa0033', 'patient code: RCa0034', 'patient code: RCa0035', 'patient code: RCa0036', 'patient code: RCa0037', 'patient code: RCa0038'], 1: ['sample type: SS', 'sample type: PB'], 2: ['origin: Brussel: UZ Brussel Oncologisch Centrum', 'origin: Candiolo: Institute for Cancer Research, University of Turin', 'origin: Turin: Deparment of Medical Science, Uni